
# 🧠 Análise de Sentimentos - Google Play Store Reviews

Este projeto tem como objetivo construir um modelo de Machine Learning para classificar avaliações de usuários como **positivas** ou **negativas** com base no conteúdo textual das reviews na Google Play Store.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re, string, joblib

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


### 🔹 Carga dos Dados

In [ ]:

url = 'https://raw.githubusercontent.com/xxjelax/playstore-sentiment-analysis/refs/heads/main/dataset/reviews.csv'
df = pd.read_csv(url)
df.head()


### 🔹 Análise Exploratória

In [ ]:

df.info()
df.isnull().sum()
sns.countplot(x='score', data=df)
plt.title('Distribuição das Avaliações')
plt.show()


### 🔹 Pré-processamento e Limpeza

In [ ]:

df.dropna(subset=['content', 'score'], inplace=True)
df = df[df['score'] != 3]
df['sentiment'] = df['score'].apply(lambda x: 1 if x >= 4 else 0)

def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

stop_words = set(stopwords.words('english'))
df['clean_content'] = df['content'].apply(clean_text)
df['clean_content'] = df['clean_content'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))


### 🔹 Separação Treino/Teste (Holdout)

In [ ]:

SEED = 42
X = df['clean_content']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED, stratify=y)


### 🔹 Etapa 8 – Modelagem com Algoritmos Clássicos (Linha de Base)

In [ ]:

baseline_models = {
    'Naive Bayes': MultinomialNB(),
    'SVM': SVC(),
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(random_state=SEED)
}

for nome, modelo in baseline_models.items():
    print(f"\n📌 Modelo (linha base): {nome}")
    pipe = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('model', modelo)
    ])
    y_pred = cross_val_predict(pipe, X_train, y_train, cv=5)
    print("Accuracy:", accuracy_score(y_train, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred))
    print("Classification Report:\n", classification_report(y_train, y_pred, target_names=["Negativo", "Positivo"]))


### 🔹 Etapa 9 – Otimização de Hiperparâmetros (GridSearchCV)

In [ ]:

modelos_grids = {
    'Naive Bayes': {
        'pipeline': Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('model', MultinomialNB())
        ]),
        'param_grid': {
            'tfidf__max_df': [0.8, 1.0],
            'tfidf__min_df': [2, 5],
            'model__alpha': [0.5, 1.0]
        }
    },
    'SVM': {
        'pipeline': Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('model', SVC())
        ]),
        'param_grid': {
            'tfidf__max_df': [0.8, 1.0],
            'tfidf__min_df': [2, 5],
            'model__C': [1, 10],
            'model__kernel': ['linear', 'rbf']
        }
    },
    'KNN': {
        'pipeline': Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('model', KNeighborsClassifier())
        ]),
        'param_grid': {
            'tfidf__max_df': [0.8, 1.0],
            'tfidf__min_df': [2, 5],
            'model__n_neighbors': [3, 5, 7]
        }
    },
    'Decision Tree': {
        'pipeline': Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('model', DecisionTreeClassifier(random_state=SEED))
        ]),
        'param_grid': {
            'tfidf__max_df': [0.8, 1.0],
            'tfidf__min_df': [2, 5],
            'model__max_depth': [10, 20, None]
        }
    }
}

modelos_treinados = {}
for nome, config in modelos_grids.items():
    print(f"\n🔍 Treinando modelo com GridSearchCV: {nome}")
    grid = GridSearchCV(config['pipeline'], config['param_grid'], cv=5, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train, y_train)
    modelos_treinados[nome] = grid
    print(f"Melhores parâmetros: {grid.best_params_}")
    print(f"Melhor score de validação cruzada: {grid.best_score_:.4f}")


### 🔹 Etapa 10 – Avaliação dos Modelos Otimizados

In [ ]:

resultados = {}
for nome, modelo in modelos_treinados.items():
    print(f"\n📌 Avaliação Final - {nome}")
    y_pred = modelo.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred, target_names=["Negativo", "Positivo"]))
    resultados[nome] = accuracy_score(y_test, y_pred)


### 🔹 Etapa 11 – Exportação do Modelo Final

In [ ]:

melhor_nome = max(resultados, key=resultados.get)
melhor_modelo = modelos_treinados[melhor_nome]
joblib.dump(melhor_modelo, 'sentiment_model.pkl')
print(f'Modelo salvo: {melhor_nome}')



### ✅ Etapa 12 – Conclusões

O modelo com melhor desempenho foi selecionado após análise de diversos algoritmos usando validação cruzada e otimização de hiperparâmetros. As métricas de precisão, recall e F1-score indicam que o modelo está apto a identificar avaliações positivas e negativas com alta confiabilidade.




### 🔗 Etapa 13 – Referências

- Dataset: [Kaggle - Google Play Store Reviews](https://www.kaggle.com/datasets/prakharrathi25/google-play-store-reviews)
- Scikit-Learn documentation: https://scikit-learn.org/
- NLTK Stopwords: https://www.nltk.org/
